In [7]:
import speech_recognition as sr
import pandas as pd
# obtain path to "english.wav" in the same folder as this script
import os 
from multiprocessing import Pool, cpu_count
from functools import partial

orig_db = pd.read_csv('csv/orig_librispeech_dev-clean.csv')
trans_root_dir = '/nas/home/cborrelli/speech_forensics/sphinx'

In [8]:
orig_db

,reader,book,librispeech_folder,path,transcription_path,SNR,noise_type
0,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
1,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
2,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
3,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
4,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
5,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
6,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
7,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
8,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN
9,2277,149896,dev-clean,/nas/public/dataset/LibriSpeech/dev-clean/2277...,/nas/public/dataset/LibriSpeech/dev-clean/2277...,inf,NaN


In [2]:
def transcribe(song_path, transcriber):
    # if file does not exist
    
    r = sr.Recognizer()
    with sr.AudioFile(song_path) as source:
        audio = r.record(source)  # read the entire audio file
    
    # here depending on transcriber argument we will switch on different methods
    if transcriber == 'sphinx':
        transcription = r.recognize_sphinx(audio)
    else:
        pass
    
    # makedirs + write txt file
    
    return [song_path, transcription]

In [ ]:
audio_files = orig_db['path'].to_list()
transcriber = 'sphinx'
f_part = partial(transcribe, transcriber=transcriber)
p = Pool(cpu_count())
a = p.map(f_part, audio_files)

t = dict(filter(None, a))


p.close()

In [20]:
root_path = '/nas/home/cborrelli/pippo'
for index, item in orig_db.iterrows():

    transcription = t[item['path']]
    dataset = item['librispeech_folder']
    reader = str(item['reader'])
    book = str(item['book'])
    name = item['path'].split('/')[-1].replace('.flac','.txt')
    path = os.path.join(root_path,transcriber,dataset,reader,book,name)
    if not os.path.exists(os.path.join(root_path,transcriber,dataset,reader)):
        os.makedirs(os.path.join(root_path,transcriber,dataset,reader))
    if not os.path.exists(os.path.join(root_path,transcriber,dataset,reader,book)):
        os.makedirs(os.path.join(root_path,transcriber,dataset,reader,book))
    if not os.path.exists(os.path.join(root_path,transcriber,dataset,reader,book,name)):
        with open(os.path.join(root_path,transcriber,dataset,reader,book,name), "w") as text_file:
            text_file.write(transcription)

    
    

In [19]:
t

{'/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0026.flac': 'the long vessel had began pedestrians internet collars and trousers at the bottom',
 '/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0005.flac': 'many little wrinkles gathered between his eyes as he contemplated as bad as that license',
 '/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0033.flac': 'then he rang the bell no answer',
 '/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0006.flac': 'he can arrange that satisfactorily for carrie would be glad to wait if necessary',
 '/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0018.flac': 'his first impulse was to read of forwards in reply go to the devil',
 '/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0034.flac': 'he made again this time harder still no answer',
 '/nas/public/dataset/LibriSpeech/dev-clean/2277/149896/2277-149896-0021.flac': 'what would she do about